In [26]:
from typing import Callable, Set, Generator, Iterable, Dict, List
import datetime
import collections
import itertools

import pymysql
import functools
from lxml import etree as et
import httplib2
from tqdm import tqdm

import helpers
import boe

In [19]:
db_host = 'localhost'
db_database = 'boe'
db_user = 'root'
db_password = 'pass'

use_db_settings = lambda func, **kwargs: func(host=db_host, user=db_user, password=db_password, database=db_database, **kwargs)

In [21]:
def f(**kwargs):
    print(kwargs)
use_db_settings(f)

{'host': 'localhost', 'user': 'root', 'password': 'pass', 'database': 'boe'}


# BOE entry label extraction

## Helper functions

In [10]:
def retrieve_entries(host: str, 
                     user: str, 
                     password: str, 
                     database: str, 
                     day: datetime.datetime) -> Iterable[Dict]:
    '''Retrieve entries for a specific day.'''
    
    connection = pymysql.connect(host=host,
                                 user=user,
                                 password=password,
                                 db=database)

    with connection:
        cursor = connection.cursor(pymysql.cursors.DictCursor)
        cursor.execute(f"SELECT * FROM boe_diary_entry WHERE date = '{day.isoformat()}';")
        entries = cursor.fetchall()
    
    return entries

def get_labels_from_tree(tree):
    tree_search = helpers.use_tree_for_search(tree)
    topics = tree_search(boe.EntryXpath.topics)
    alerts = tree_search(boe.EntryXpath.alerts)
    
    labels = helpers.pipe(topics + alerts,
        functools.partial(map, lambda x: x.xpath('text()')),
        functools.partial(map, ''.join),
        set)
    return labels

def get_tree(item):
    response = helpers.fetch_page('https://www.boe.es' + item.get('xml_url'))
    tree = helpers.tree_from_response(response)
    
    return tree

def labels_insert_queries(entry_id: str, labels: Iterable[str]) -> List[str]:
    def single_insert_query(entry_id: str, label: str) -> str:
        return f'''INSERT INTO boe_diary_entry_label (entry, name)
            VALUES ('{entry_id}', '{label}')
        '''
    
    queries = map(functools.partial(single_insert_query, entry_id), labels)
    return queries

def insert_labels(use_db_settings: Callable, entry_id: str, labels: Iterable[str]):
    if len(labels) == 0:
        return
    
    connection = use_db_settings(pymysql.connect)

    with connection:
        cursor = connection.cursor()
        
        for query in labels_insert_queries(entry_id, labels):
            cursor.execute(query)

## Execution

In [22]:
DATE = datetime.datetime(2021, 1, 26)

items = use_db_settings(retrieve_entries, day=DATE)

for item in tqdm(items):
    tree = get_tree(item)
    labels = get_labels_from_tree(tree)
    insert_labels(use_db_settings, item.get('id'), labels)

100%|██████████| 332/332 [00:52<00:00,  6.30it/s]


In [25]:
collections.Counter()

Counter({'1': 2, '2': 2, '3': 3, '5': 2, '4': 2, '6': 1})

In [32]:
title_c = collections.Counter(itertools.chain(*(item.get('title').split() for item in items)))

In [48]:
epigraph_c = collections.Counter(itertools.chain((item.get('epigraph') for item in items)))
epigraph_c

Counter({'Medidas urgentes': 1,
         'Organización': 2,
         'Nombramientos': 8,
         'Destinos': 9,
         'Personal directivo': 1,
         'Ceses': 1,
         'Letrados al servicio del Tribunal Supremo': 1,
         'Cuerpo de funcionarios docentes': 1,
         'Personal funcionario y laboral': 1,
         'Agencia Española de Cooperación Internacional para el Desarrollo. Cuentas anuales': 1,
         'Convenios': 11,
         'Comunidad Foral de Navarra. Convenio': 1,
         'Delegación de competencias': 1,
         'Sector eléctrico': 1,
         'Deuda del Estado': 1,
         'Cargos representativos locales': 1,
         'Mercado de divisas': 1,
         'Agencias de valores': 1,
         'Competencias profesionales': 1,
         'Sentencias': 19,
         'Autos': 2,
         '': 266})

In [50]:
department_c = collections.Counter(itertools.chain((item.get('department') for item in items)))
department_c

Counter({'JEFATURA DEL ESTADO': 1,
         'MINISTERIO DE EDUCACIÓN Y FORMACIÓN PROFESIONAL': 6,
         'MINISTERIO DE TRABAJO Y ECONOMÍA SOCIAL': 1,
         'CONSEJO GENERAL DEL PODER JUDICIAL': 5,
         'MINISTERIO DE ASUNTOS EXTERIORES, UNIÓN EUROPEA Y COOPERACIÓN': 3,
         'MINISTERIO DE HACIENDA': 26,
         'MINISTERIO DEL INTERIOR': 1,
         'MINISTERIO DE TRANSPORTES, MOVILIDAD Y AGENDA URBANA': 5,
         'MINISTERIO DE INDUSTRIA, COMERCIO Y TURISMO': 1,
         'MINISTERIO DE POLÍTICA TERRITORIAL Y FUNCIÓN PÚBLICA': 4,
         'MINISTERIO DE CULTURA Y DEPORTE': 2,
         'MINISTERIO DE ASUNTOS ECONÓMICOS Y TRANSFORMACIÓN DIGITAL': 3,
         'COMUNIDAD AUTÓNOMA DE GALICIA': 2,
         'ADMINISTRACIÓN LOCAL': 2,
         'MINISTERIO DE JUSTICIA': 2,
         'MINISTERIO DE DEFENSA': 11,
         'MINISTERIO DE LA PRESIDENCIA, RELACIONES CON LAS CORTES Y MEMORIA DEMOCRÁTICA': 6,
         'MINISTERIO PARA LA TRANSICIÓN ECOLÓGICA Y EL RETO DEMOGRÁFICO': 10,

In [51]:
sum(department_c.values())

332

In [37]:
max(title_c.values())

1108

In [39]:
[word for word, count in title_c.items() if count == 6]

['enero,',
 'Junta',
 'Comisión',
 'Acuerdo',
 'nombra',
 'Seguridad',
 'Consejería',
 'Artes',
 '20',
 'Estado',
 'ejercicio',
 'Social',
 'Segunda.',
 'libertad',
 'Lorca',
 'efectiva:',
 '40/2020',
 '[inadecuada',
 'utilización',
 'dirección',
 'electrónica',
 'habilitada',
 'cauce',
 'comunicación',
 '(SSTC',
 '6/2019',
 '47/2019)',
 'oposición',
 'confusión',
 'jurídicas',
 'relacionarse',
 'comunicaciones',
 'electrónicas',
 'civiles].',
 'Investigaciones',
 'Científicas,',
 'M.P.',
 'Facultad',
 'Costas']

In [40]:
items[0]

{'id': 'BOE-A-2021-1064',
 'date': datetime.date(2021, 1, 26),
 'title': 'Corrección de errores del Real Decreto-ley 36/2020, de 30 de diciembre, por el que se aprueban medidas urgentes para la modernización de la Administración Pública y para la ejecución del Plan de Recuperación, Transformación y Resiliencia.',
 'section': '1',
 'department': 'JEFATURA DEL ESTADO',
 'epigraph': 'Medidas urgentes',
 'pdf_url': '/boe/dias/2021/01/26/pdfs/BOE-A-2021-1064.pdf',
 'xml_url': '/diario_boe/xml.php?id=BOE-A-2021-1064',
 'htm_url': '/diario_boe/txt.php?id=BOE-A-2021-1064',
 'has_economic_impact': None}

In [23]:
for item in items:
    tree = get_tree(item)
    labels = get_labels_from_tree(tree)
    print(item.get('title'))
    print(labels)

Corrección de errores del Real Decreto-ley 36/2020, de 30 de diciembre, por el que se aprueban medidas urgentes para la modernización de la Administración Pública y para la ejecución del Plan de Recuperación, Transformación y Resiliencia.
{'Administración electrónica', 'Política económica', 'Epidemias', 'Industria', 'Medio ambiente', 'Unión Europea', 'Ayudas', 'Obras y construcciones', 'Administraciones Públicas', 'Contrato de concesión de obras públicas', 'Instituto de Comercio Exterior', 'Derecho Administrativo', 'Autorizaciones', 'Organización de la Administración', 'Evaluación de impacto ambiental', 'Procedimiento administrativo', 'Agencias estatales', 'Tecnología e investigación', 'Presupuestos Generales del Estado', 'Organización de la Administración del Estado'}
Orden EFP/38/2021, de 21 de enero, por la que se modifica la Orden EFP/1418/2018, de 27 de diciembre, por la que se crea la Junta de Contratación y se constituye la Mesa de Contratación del Ministerio de Educación y Form

Resolución de 14 de enero de 2021, de la Secretaría de Estado de Justicia, por la que se publica el Convenio con la Fundación Universitat Pompeu Fabra, para la realización de prácticas en el Departamento de Barcelona del Instituto Nacional de Toxicología y Ciencias Forenses.
set()
Resolución 420/38660/2020, de 17 de noviembre, de la Secretaría General Técnica, por la que se publica el Convenio con la Real Hermandad de Veteranos de las Fuerzas Armadas y de la Guardia Civil, para la realización de actividades de voluntariado en el Hospital Central de la Defensa Gómez Ulla.
set()
Orden EFP/43/2021, de 21 de enero, sobre fijación de límites para la administración de determinados créditos para gastos y de delegación de competencias.
{'Dirección General de Planificación y Gestión Educativa', 'Secretarías generales técnicas ministeriales', 'Dirección General de Evaluación y Cooperación Territorial', 'Centros de enseñanza españoles en el extranjero', 'Direcciones provinciales ministeriales', '

Sala Segunda. Sentencia 182/2020, de 14 de diciembre de 2020. Recurso de amparo 1192-2019. Promovido por don Sergio Linacero Fajardo en relación con el decreto dictado por la letrada de la administración de justicia de un juzgado de lo penal de Valencia en procedimiento de ejecutoria. Vulneración del derecho a la tutela judicial efectiva (acceso a la jurisdicción): resolución que, en aplicación del precepto anulado por la STC 151/2020, de 22 de octubre, impide la revisión judicial de decisiones adoptadas por la letrada de la administración de justicia.
set()
Sala Primera. Sentencia 183/2020, de 14 de diciembre de 2020. Recurso de amparo 3096-2019. Promovido por Penrei Inversiones, S.L., respecto de las resoluciones dictadas por un juzgado de primera instancia e instrucción de Lorca en procedimiento de ejecución hipotecaria. Vulneración del derecho a la tutela judicial efectiva: STC 40/2020 [inadecuada utilización de la dirección electrónica habilitada como cauce de comunicación del pri

Pleno. Sentencia 193/2020, de 17 de diciembre de 2020. Recurso de amparo 5198-2019. Promovido por don Jordi Turull i Negre respecto de los acuerdos de la mesa del Congreso de los Diputados declarando su suspensión como diputado. Supuesta vulneración de los derechos al ejercicio del cargo público representativo y a la presunción de inocencia: STC 97/2020 [resoluciones parlamentarias que aplican la previsión legal de suspensión automática del ejercicio del cargo parlamentario (STC 71/1994)].
set()
Pleno. Sentencia 194/2020, de 17 de diciembre de 2020. Recurso de amparo 5199-2019. Promovido por don Josep Rull i Andreu respecto de los acuerdos de la mesa del Congreso de los Diputados declarando su suspensión como diputado. Supuesta vulneración de los derechos al ejercicio del cargo público representativo y a la presunción de inocencia: STC 97/2020 [resoluciones parlamentarias que aplican la previsión legal de suspensión automática del ejercicio del cargo parlamentario (STC 71/1994)].
set()

Anuncio de la Confederación Hidrográfica del Guadalquivir de desistimiento del procedimiento de adjudicación del "contrato de obra de renovación integral de elementos de valvulería, electromecánicos y otros de la presa del Quéntar (Granada). GR(DT)-5997".
set()
Anuncio de licitación de: Gerencia de Atención Sanitaria de Melilla - Instituto Nacional de Gestión Sanitaria. Objeto: Servicio de transporte sanitario en Melilla de enfermos a los que el INGESA tenga el deber legal o convencional de trasladar en ambulancia. Expediente: P.A. 2020-12-GME.
set()
Anuncio de formalización de contratos de: Subdirección General de Análisis Presupuestario y Gestión Financiera del Instituto de Mayores y Servicios Sociales. Objeto: Servicio de reserva y ocupación de plazas residenciales y de centro de día para personas dependientes con discapacidad intelectual en Ceuta. Expediente: 317/2020.
set()
Anuncio de licitación de: Presidencia de la Agencia Estatal Consejo Superior de Investigaciones Científicas,

Anuncio del Servicio Provincial de Costas en Lugo referente a la Información pública de la solicitud de autorización para impartir cursos de surf en la playa de Lóngara, T.M. Barreirros (Lugo).
set()
Anuncio de la Demarcación de Costas en Cantabria sobre solicitud de autorización de servicios de temporada en playas del término municipal de Alfoz de Lloredo. Peticionario: Ayuntamiento de Alfoz de Lloredo. Expediente AUT01/21/39/0006.
set()
Anuncio del Servicio Provincial de Costas en Lugo referente a la Información pública de la solicitud de autorización para impartir cursos de surf en la playa de A Rapadoira, T.M. Foz (Lugo).
set()
Anuncio del Servicio Provincial de Costas en Lugo referente a la Información pública de la solicitud de autorización para instalación de terraza en Plaza Calvo Sotelo y Bustelo, 2 – Rinlo, T.M. Ribadeo (Lugo).
set()
Anuncio de la Demarcación de Costas en Asturias sobre inicio de expediente de autorización de ocupación del dominio público marítimo-terrestre d